In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import random
import functools
from functools import reduce

In [2]:
image_size = 28
input_num_units=28*28
num_labels = 10
output_num_units=num_labels
learning_rate=0.5
num_steps =501
num_train_samples = 10000
beta=0.01

In [3]:
pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)#Concise 1-Hot Encoding in NUMPY
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def getSampleIndexes(population,sample_sz):
    if(sample_sz>population.shape[0]):
        return np.array(range(population.shape[0]))
    rand_indexes=random.sample(range(population.shape[0]),sample_sz)
    return population[rand_indexes,:],rand_indexes
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])
def isEqual(nparr1,nparr2):
    totDims=len(nparr1.shape)
    if np.sum([nparr1.shape[i]==nparr2.shape[i] for i in range(totDims)])!=totDims:
        return False
    eqItems=nparr1==nparr2
    if np.sum(eqItems)==reduce(lambda x,y:x*y,nparr1.shape):
        return True
    else:
        return False
def apply_reg_opt(weight,bias,train_x,train_y,regularizer):
    weighted_sum = tf.matmul(train_x, weight) + bias 
    cost=tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=weighted_sum)
    cost=cost+regularizer
    loss_fun = tf.reduce_mean(cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_fun)
    return optimizer,loss_fun
def predict_values(x,weights,biases):
    return tf.nn.softmax(tf.matmul(x,weights)+biases)

In [6]:
#Logistic Regression : Declaring the Model as Computational Graph
graph_reg = tf.Graph()
with graph_reg.as_default():
    train_dataset_tf = tf.placeholder(tf.float32, [None, input_num_units])
    train_labels_tf = tf.placeholder(tf.float32, [None, num_labels])
    valid_dataset_tf = tf.placeholder(tf.float32, [None, input_num_units])
    test_dataset_tf = tf.placeholder(tf.float32, [None, input_num_units])
    
    weights_l2 = tf.Variable(tf.truncated_normal([input_num_units, num_labels]))
    biases_l2 = tf.Variable(tf.zeros([num_labels]))
    l2_penalty=beta*tf.nn.l2_loss(weights_l2)
    optimizer_l2,loss_fun=apply_reg_opt(weights_l2,biases_l2,train_dataset_tf,train_labels_tf,l2_penalty)
    
    train_prediction_l2 = predict_values(train_dataset_tf,weights_l2,biases_l2)
    valid_prediction_l2 = predict_values(valid_dataset_tf,weights_l2,biases_l2)
    test_prediction_l2 = predict_values(test_dataset_tf,weights_l2,biases_l2)

In [7]:
def getFeedDict(train_x,train_y):
        return {train_dataset_tf:train_x,train_labels_tf:train_y,valid_dataset_tf:valid_dataset,
              test_dataset_tf:test_dataset}
def validate_tf_comput_graph(session,tf_feed_dict):
    _=session.run([optimizer_l2],feed_dict=tf_feed_dict)
    train_pred_1=session.run(train_prediction_l2,feed_dict=tf_feed_dict)
    train_pred_2=session.run(train_prediction_l2,feed_dict=tf_feed_dict)
    if isEqual(train_pred_1,train_pred_2):
        print("Computational Graph : Validation Passed!")
    else:
        print("Computational Graph : Validation Failed!")
with tf.Session(graph=graph_reg) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    train_x,rand_indexes=getSampleIndexes(train_dataset,num_train_samples)
    train_y=train_labels[rand_indexes]
    validate_tf_comput_graph(session,getFeedDict(train_x,train_y))
    for step in range(num_steps):
        train_x,rand_indexes=getSampleIndexes(train_dataset,num_train_samples)
        train_y=train_labels[rand_indexes]
        _,loss,train_pred_l2=session.run([optimizer_l2,loss_fun, train_prediction_l2],
                                      feed_dict=getFeedDict(train_x,train_y))
        if step%100==0:
            print('Loss at step %d: %f' % (step, loss))
            valid_pred_l2,test_pred_l2=session.run([valid_prediction_l2,test_prediction_l2],
                                                   feed_dict=getFeedDict(train_x,train_y))
            train_acc_l2=accuracy(train_pred_l2, train_y)
            valid_acc_l2=accuracy(valid_pred_l2, valid_labels)
            test_acc_l2=accuracy(test_pred_l2,test_labels)
            print("Accuracy ~~~~~~~~~~~~>",train_acc_l2,valid_acc_l2,test_acc_l2)
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

Initialized
Computational Graph : Validation Passed!
Loss at step 0: 43.515495
Accuracy ~~~~~~~~~~~~> 14.78 18.27 19.26
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 100: 11.849002
Accuracy ~~~~~~~~~~~~> 73.3 72.7 81.0
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 200: 4.498791
Accuracy ~~~~~~~~~~~~> 77.99 76.79 85.4
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 300: 1.995128
Accuracy ~~~~~~~~~~~~> 80.67 79.63 88.02
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 400: 1.162605
Accuracy ~~~~~~~~~~~~> 82.11 80.77 89.02
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 500: 0.875537
Accuracy ~~~~~~~~~~~~> 82.31 81.25 89.47
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
